In [ ]:
import numpy as np
import import_ipynb
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import load_iris
from tensorflow.python.keras.layers import Input, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
from tensorflow.keras import layers

class Layer:
    def __init__(self, input_shape=None, neurons=2, activation="relu"):
        self.input_shape = input_shape
        self.neurons = neurons
        self.activation = activation
        self.prev = None
        self.next = None

    def Dense(self):
        return layers.Dense(self.neurons, activation=self.activation)

In [ ]:
class headers:
    def __init__(self):
        self.next = None
        self.prev = None

class MLP:
    def __init__(self,model_input=None,model_output=None,num_of_classes=None,num_of_layers=1,activation='relu'):
        self.model_input= model_input
        self.model_output = model_output
        self.num_of_classes = num_of_classes
        self.num_of_layers = num_of_layers
        self.activation  = activation
        self.head = headers()
        self.tail = headers()
        self.model =  tf.keras.Sequential([]) # recommended to build the model in tf_build()
        self.init_layer = None
        self.output_layer = None

        self._built = False        # stop it from rebuilding over and over
        self._compiled = False

        self.input_dim = None
        if hasattr(model_input, "shape") and len(model_input.shape) >= 2:
        # infer feature dim from second axis (works for arrays/tensors shaped (N, D))
            try:
                self.input_dim = int(model_input.shape[1])
            except Exception:
                self.input_dim = None

    def build_network(self):

        if self._built:
            return   # once it's built it won't rebuild

        if self.num_of_classes is None and self.model_output is not None: # identify number of classes
            self.num_of_classes = np.unique(self.model_output).shape[0]

        init_layer = Layer(neurons=10,activation='relu',input_shape=self.model_input.shape)

        init_layer.prev = self.head
        self.head.next = init_layer
        init_layer.next = self.tail
        self.tail.prev = init_layer
        self.init_layer = init_layer
        for i in range(self.num_of_layers-1):
            temp_layer = Layer(neurons=10)
            self.tail.prev.next = temp_layer
            temp_layer.prev = self.tail.prev
            self.tail.prev = temp_layer
            temp_layer.next = self.tail
        if self.num_of_classes is not None:
            output_layer = Layer(neurons=self.num_of_classes,activation='softmax')
            last_hidden = self.tail.prev
            last_hidden.next = output_layer
            output_layer.prev = last_hidden
            output_layer.next = self.tail
            self.tail.prev = output_layer
            self.output_layer = output_layer   # reference

        self._built = True 

    def add_layer(self,neurons,activation):
        temp_layer = Layer(neurons=neurons,activation=activation)
        self.output_layer.prev.next = temp_layer
        temp_layer.layer.prev = self.output_layer.prev
        self.output_layer.prev = temp_layer
        temp_layer.next = self.output_layer

    def hop(self,steps):

        temp = self.head
        for i in range(steps):
            temp = temp.next
        return temp

    def edit_layer(self,layer_number,neurons=2,activation='relu'):

        temp_layer = self.hop(layer_number)
        new_layer = Layer(neurons=neurons,activation=activation)
        temp_layer.prev.next = new_layer
        temp_layer.next.prev = new_layer

    def tf_build(self):
        if self.model.layers:
            return self.model   # in case of already built

        network = self.build_network()
        #self.model  = tf.keras.Sequential([])
        model = tf.keras.Sequential([])
        print(model)
        temp = self.init_layer
        while temp.next is not self.tail:

            model.add(temp.Dense())  # sus
            temp = temp.next

        self.model = model
        return self.model

    def compile(self, optimizer="adam", loss=None, metrics=None):

        if self._compiled:
            return   # compile() can no longer be called multiple times

    # build model (tf_build sets self.model)
        self.tf_build()

    # choose correct default loss
        if loss is None:
            if getattr(self, "output_layer", None) is not None \
               and getattr(self.output_layer, "activation", "") == "softmax":
                loss = "categorical_crossentropy"
            else:
                loss = "mse"

        self.model.compile(
            optimizer=optimizer,
            loss=loss,
            metrics=metrics or []
        )

        self._compiled = True   # DONE

    def fit(self, X, y, **kwargs):
        if not self._compiled:
            raise RuntimeError("Call compile() before fit()")  # ERR: can't call fit before compiled now
            
        #model  = self.tf_build()
        #model.compile(self.model.fit(X, y, **kwargs))
        return self.model.fit(X, y, **kwargs)

    def predict(self, X, **kwargs):
        model  = self.tf_build()
        return self.model.predict(X, **kwargs)

    def summary(self):
        model  = self.tf_build()
        return self.model.summary()

    def evaluate():
        pass

In [ ]:
iris = load_iris()
X = iris.data
print(X.shape)
y = iris.target
print(X.shape)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
model = MLP(model_input=X_train,model_output=y_train,num_of_layers=5,activation='softmax')
network = model.build_network()
print(model.model)
model.compile(optimizer="adam", metrics=["accuracy"],loss="sparse_categorical_crossentropy")
model.fit(X_train,y_train)
model.predict(X_test)
model.summary()
network
model.tail
model.output_layer.neurons